In [6]:
# import pandas
import pandas as pd
import sqlalchemy
print(sqlalchemy.__version__)

1.3.18


In [2]:
# read clicks table into pandas dataframe
clicks_df = pd.read_parquet('ds_clicks.parquet.gzip')

In [3]:
# read clicks table into pandas dataframe
leads_df = pd.read_parquet('ds_leads.parquet.gzip')

In [4]:
# read offers into pandas dataframe
offers_df = pd.read_parquet('ds_offers.parquet.gzip')

### Create a connection to the database


In [5]:
import sqlite3
from sqlalchemy import create_engine


#Connecting to sqlite
conn = sqlite3.connect('loan_offers.db')

#Creating a cursor object using the cursor() method
cursor = conn.cursor()


### Create clicks table using sqlite3

In [6]:
clicks_df.head(1) # display first row

,offer_id,clicked_at
0,810116813,2021-03-23 02:01:48.339


In [7]:
clicks_df.shape # display the shape

(9320, 2)

In [8]:
cursor.execute("DROP TABLE IF EXISTS clicks")

clicks_table ='''CREATE TABLE clicks(
   offer_id int PRIMARY KEY NOT NULL,
   clicked_at DateTime
)'''
cursor.execute(clicks_table)

print("Table created successfully........")

# Commit the changes in the database
conn.commit()

Table created successfully........


In [9]:
# INSERT records into the database.
for index, row in clicks_df.iterrows():
    insert = f"INSERT INTO clicks (offer_id, clicked_at) VALUES('{row.offer_id}', '{row.clicked_at}')"
    cursor.execute(insert)

    
conn.commit()
print("Records inserted........")

# Closing the connection

Records inserted........


In [10]:
#Retrieving data
cursor.execute('''SELECT * from clicks''')

#Fetching 1st row from the table
result = cursor.fetchone();
print(result)


(810116813, '2021-03-23 02:01:48.339000')


### Create leads table using sqlite3

In [11]:
leads_df.head(1) # display first row

,lead_uuid,requested,loan_purpose,credit,annual_income
0,004cfd11-4d3f-4ba2-ad7d-0c0bb215e1f6,700.0,debt_consolidation,poor,24000.0


In [12]:
cursor.execute("DROP TABLE IF EXISTS leads")

leads_table ='''CREATE TABLE leads(
   lead_uuid varchar PRIMARY KEY NOT NULL,
   requested float,
   loan_purpose text,
   credit text,
   annual_income float
)'''
cursor.execute(leads_table)

print("Table created successfully........")

# Commit the changes in the database
conn.commit()

Table created successfully........


In [13]:
# INSERT a records into the database.
for index, row in leads_df.iterrows():
    insert = f"INSERT INTO leads (lead_uuid, requested, loan_purpose, credit, annual_income)\
    VALUES('{row.lead_uuid}', '{row.requested}','{row.loan_purpose}','{row.credit}','{row.annual_income}')"
    cursor.execute(insert)

    
conn.commit()
print("Records inserted........")

# Closing the connection

Records inserted........


In [14]:
#Retrieving data
cursor.execute('''SELECT * from leads''')

#Fetching 1st row from the table
result = cursor.fetchone();
print(result)

#Fetching 1st row from the table
# result = cursor.fetchall();
# print(result)

('004cfd11-4d3f-4ba2-ad7d-0c0bb215e1f6', 700.0, 'debt_consolidation', 'poor', 24000.0)


### Create offers table using sqlite3

In [15]:
offers_df.head(1) # Display first row

,lead_uuid,offer_id,apr,lender_id
33,ae2d5046-a7c7-44fe-b6f4-cde3d8bf29e2,810117850,199.0,1103


In [16]:
cursor.execute("DROP TABLE IF EXISTS offers")

offers_table ='''CREATE TABLE offers(
   offer_id int PRIMARY KEY NOT NULL,
   lead_uuid varchar,
   apr float,
   lender_id int
)'''
cursor.execute(offers_table)

print("Table created successfully........")

# Commit your changes in the database
conn.commit()

Table created successfully........


In [17]:
# INSERT records into the database.
for index, row in offers_df.iterrows():
    insert = f"INSERT INTO offers (offer_id,lead_uuid, apr, lender_id)\
    VALUES('{row.offer_id}', '{row.lead_uuid}','{row.apr}','{row.lender_id}')"
    cursor.execute(insert)

    
conn.commit()
print("Records inserted........")

# Closing the connection

Records inserted........


In [18]:
#Retrieving data
cursor.execute('''SELECT * from offers''')

#Fetching 1st row from the table
result = cursor.fetchone();
print(result)

#Fetching 1st row from the table
# result = cursor.fetchall();
# print(result)

(810117850, 'ae2d5046-a7c7-44fe-b6f4-cde3d8bf29e2', 199.0, 1103)


In [20]:
# load data from database
engine = create_engine('sqlite:///loan_offers.db')
# read offers table
offers = pd.read_sql_table("offers", con=engine)
offers.head()

,offer_id,lead_uuid,apr,lender_id
0,810117850,ae2d5046-a7c7-44fe-b6f4-cde3d8bf29e2,199.00,1103
1,810119030,b12fbb06-1402-4de3-a91f-fb6360ff85e4,249.00,1103
2,810122970,a119a3db-ab14-46fc-acd1-35cf20dec1ec,249.00,1103
3,810124218,3166d6bd-1c79-44c0-867c-889afd35990c,17.69,240
4,810124220,3166d6bd-1c79-44c0-867c-889afd35990c,17.19,240


In [21]:
# display length of offers table
len(offers) 

458939

In [22]:
# display unique values of "offer_id"
offers['offer_id'].nunique()

458939

In [23]:
# read clicks table
clicks = pd.read_sql_table("clicks", con=engine)
clicks.head()

,offer_id,clicked_at
0,810116813,2021-03-23 02:01:48.339
1,810118339,2021-03-23 02:01:14.135
2,810132429,2021-03-23 02:46:49.753
3,810152009,2021-03-23 04:46:19.662
4,810177207,2021-03-23 08:44:04.494


In [24]:
# read leads table
leads = pd.read_sql_table("leads", con=engine)
leads.head()

,lead_uuid,requested,loan_purpose,credit,annual_income
0,004cfd11-4d3f-4ba2-ad7d-0c0bb215e1f6,700.0,debt_consolidation,poor,24000.0
1,00aa9e4d-52c9-4240-864d-442646a422cc,1500.0,auto,poor,60000.0
2,00b02fa9-84cb-4219-abe8-29a35250a5d7,1000.0,other,good,60000.0
3,00e1153a-32e8-4cc3-9b3f-fe395ba855e2,1000.0,debt_consolidation,fair,36000.0
4,011f9ddb-398a-4488-b704-ba49743949ab,3000.0,debt_consolidation,fair,79200.0


In [25]:
len(leads)

100000

Our goal is to build a machine learing model to predict whether an offer is going to be clicks. So we will use the offers table left to left join with leads table - this will return all rows from the offers table, and the matching rows from the leads table. 

After left join is done, we will then left join again with the clicks table.

In [26]:
# Use offers to left join with leads
merged_df = offers.merge(leads, on='lead_uuid', how='left')

In [27]:
# Leften join again with clicks table
merged_df = merged_df.merge(clicks, on='offer_id', how='left')

In [33]:
# Dispaly random 5 rows of the new table
merged_df.sample(5)

,offer_id,lead_uuid,apr,lender_id,requested,loan_purpose,credit,annual_income,clicked_at
166113,811411089,06074d42-3482-4645-a9cb-aaa3ccababb3,28.13,262,18000.0,credit_card_refi,excellent,74000.0,NaT
317301,811576316,d12cb7a3-f5b3-4a42-9f4a-38d3234b2eca,149.00,1777,1000.0,debt_consolidation,poor,27000.0,NaT
414214,812563683,3e5fd35a-94f9-4e96-aada-8a43a74380f1,149.00,1777,2000.0,auto,fair,42000.0,NaT
458541,813041194,c83a4487-b259-45d2-8b33-fc1c1582622d,35.99,327,1000.0,debt_consolidation,excellent,36000.0,NaT
119662,810926373,305d81b1-eb6c-4f01-b500-396acb0e225b,27.09,363,3000.0,special_occasion,fair,46000.0,NaT


In [29]:
# Display table length
len(merged_df)

458939

### Save table to database

In [30]:
# Save merged table to merged.db
engine = create_engine('sqlite:///merged.db', encoding = 'UTF-8')
merged_df.to_sql('merged', engine, if_exists='replace', index=False)

In [32]:
conn.close()